In [200]:
import pandas as pd 
from scipy.stats import norm
import numpy as np 

In [202]:
time_period = '5_10'

In [203]:
data_FR = pd.read_csv('results_als_FR.csv',sep=';')
data_SW = pd.read_csv('results_als_SW.csv',sep=';')
data_AUS = pd.read_csv('results_als_AUS.csv',sep=';')

In [ ]:
data_SW = data_SW.rename(columns={'oddratio_ci':'final_odds_ratio_2_5_als','oddratio_ci.1':'final_odds_ratio_5_10_als'})


In [206]:
def prepare_df_AUS(df,country,time_period,nb_rows): 
    df['odds_ratio_'+time_period+'_'+country] = df['final_odds_ratio_'+time_period+'_als']
    #df['odds_ratio_'+time_period+'_low'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[0])
    #df['odds_ratio_'+time_period+'_high'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[1][:-1])
    #df = df.replace('<0.001 ','0.0')
    #df = df.replace('<0.001','0.0')
    #df = df.astype(dtype={'odds_ratio_'+time_period+'_'+country:'float','odds_ratio_'+time_period+'_low'+'_'+country:'float','odds_ratio_'+time_period+'_high'+'_'+country:'float'})
    #counf_interval = norm.ppf(1 - 0.025/nb_rows)
    df['std_'+time_period+'_'+country] = df['std_'+time_period]
    counf_interval_95 = norm.ppf(1 - 0.025)

    df['odds_ratio_interval95_'+time_period+'_'+country] = df.apply(lambda x : str(x['odds_ratio_'+time_period+'_'+country])+'('+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])-counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])+counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+')',  axis=1)
    return df

def prepare_df_FR(df,country,time_period,nb_rows): 
    df['odds_ratio_'+time_period+'_'+country] = np.exp(df['coef_'+time_period])
    #df['odds_ratio_'+time_period+'_low'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[0])
    #df['odds_ratio_'+time_period+'_high'+'_'+country] = df['final_odds_ratio_'+time_period+'_alz'].apply(lambda x : x.split('(')[1].split(',')[1][:-1])
    #df = df.replace('<0.001 ','0.0')
    #df = df.replace('<0.001','0.0')
    #df = df.astype(dtype={'odds_ratio_'+time_period+'_'+country:'float','odds_ratio_'+time_period+'_low'+'_'+country:'float','odds_ratio_'+time_period+'_high'+'_'+country:'float'})
    #counf_interval = norm.ppf(1 - 0.025/nb_rows)
    df['std_'+time_period+'_'+country] = df['std_'+time_period]
    counf_interval_95 = norm.ppf(1 - 0.025)

    df['odds_ratio_interval95_'+time_period+'_'+country] = df.apply(lambda x : str(x['odds_ratio_'+time_period+'_'+country])+'('+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])-counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])+counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+')',  axis=1)
    return df

def prepare_df_SW(df,country,time_period,nb_rows): 
    df['odds_ratio_'+time_period+'_'+country] = df['final_odds_ratio_'+time_period+'_als'].apply(lambda x : x.split('(')[0])
    df['odds_ratio_'+time_period+'_low'+'_'+country] = df['final_odds_ratio_'+time_period+'_als'].apply(lambda x : x.split('(')[1].split('-')[0])
    df['odds_ratio_'+time_period+'_high'+'_'+country] = df['final_odds_ratio_'+time_period+'_als'].apply(lambda x : x.split('(')[1].split('-')[1][:-1])
    df = df.replace('<0.001 ','0.0')
    df = df.replace('<0.001','0.0')
    df = df.astype(dtype={'odds_ratio_'+time_period+'_'+country:'float','odds_ratio_'+time_period+'_low'+'_'+country:'float','odds_ratio_'+time_period+'_high'+'_'+country:'float'})
    counf_interval = norm.ppf(1 - 0.025/nb_rows)
    df['std_'+time_period+'_'+country] = (np.log(df['odds_ratio_'+time_period+'_high'+'_'+country])-np.log(df['odds_ratio_'+time_period+'_'+country]))/(counf_interval)
    counf_interval_95 = norm.ppf(1 - 0.025)

    df['odds_ratio_interval95_'+time_period+'_'+country] = df.apply(lambda x : str(x['odds_ratio_'+time_period+'_'+country])+' ('+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])-counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+time_period+'_'+country])+counf_interval_95*x['std_'+time_period+'_'+country]).round(2))+')',  axis=1)
    return df
    

In [207]:
data_SW = prepare_df_SW(data_SW,'SW',time_period,52)
data_AUS = prepare_df_AUS(data_AUS,'AUS',time_period,1)
data_FR = prepare_df_FR(data_FR,'FR',time_period,1)

In [209]:
data_FR['n_'+time_period+'_FR2'] = data_FR['n_'+time_period+'_FR']

data_FR['n_'+time_period+'_FR'] = data_FR['n_'+time_period+'_FR'].apply(lambda x : x.split('(')[1][:-2])


df_final = pd.merge(data_AUS,data_SW,on='code',how='outer')
df_final = pd.merge(df_final,data_FR,on='code',how='outer')
df_final = df_final.rename(columns={'medoc_y':'medoc'})

mapping = pd.read_csv('mapping.csv',sep=';')
df_final = pd.merge(df_final,mapping,on='code',how='inner')
df_final['final_name'] = df_final['code']+': '+df_final['name_medoc']

In [ ]:
df_final = df_final[['medoc','code','final_name','odds_ratio_'+time_period+'_SW', 'std_'+time_period+'_SW',
            'odds_ratio_'+time_period+'_AUS', 'std_'+time_period+'_AUS',
            'odds_ratio_'+time_period+'_FR', 'std_'+time_period+'_FR', 'n_'+time_period+'_FR','n_'+time_period+'_SW','n_'+time_period+'_AUS',
                    ]]
for column in df_final.columns : 
        df_final = df_final.rename(columns={column: column.replace('_'+time_period,'')})
df_final_R = df_final.dropna().reset_index(drop=True)
df_final_R

In [ ]:
def prepare_for_R(df,time_period): 
    for column in df.columns : 
        df = df.rename(columns={column: column.replace('_'+time_period,'')})
    df['coeff_AUS'] = np.log(df['odds_ratio_AUS'])
    df['coeff_FR'] = np.log(df['odds_ratio_FR'])
    df['coeff_SW'] = np.log(df['odds_ratio_SW'])    
    return df

df_final_R = prepare_for_R(df_final_R,time_period)
df_final_R.to_csv('df_'+time_period+'_als_final.csv')
df_final_R

In [ ]:
meta_analysis = pd.read_csv('result_meta_analysis_als_R_'+time_period+'.csv')
meta_analysis = pd.merge(meta_analysis,df_final, on ='code',how='outer')
meta_analysis


In [ ]:
data_AUS['n_'+time_period+'_AUS'] = data_AUS.apply(lambda x : str(x['n_brut_'+time_period+'_AUS'])+' ('+str(np.round(x['n_'+time_period+'_AUS'],2))+'%)',axis=1)
data_AUS['n_'+time_period+'_AUS_cont'] = data_AUS.apply(lambda x : str(x['n_brut_'+time_period+'_AUS_cont'])+' ('+str(np.round(x['n_'+time_period+'_AUS_cont'],2))+'%)',axis=1)
data_AUS_n = data_AUS[['code','n_'+time_period+'_AUS','n_'+time_period+'_AUS_cont']]
data_SW['n_'+time_period+'_SW'] = data_SW.apply(lambda x : str(x['n_brut_'+time_period+'_SW'])+' ('+str(np.round(x['n_'+time_period+'_SW'],2))+'%)',axis=1)
data_SW['n_'+time_period+'_SW_cont'] = data_SW.apply(lambda x : str(x['n_brut_'+time_period+'_SW_cont'])+' ('+str(np.round(x['n_'+time_period+'_SW_cont'],2))+'%)',axis=1)
data_SW_n = data_SW[['code','n_'+time_period+'_SW','n_'+time_period+'_SW_cont']]
data_FR_n = data_FR[['code','n_'+time_period+'_FR2','n_'+time_period+'_FR_cont']]
data_AUS_n = data_AUS[['code','n_'+time_period+'_AUS','n_'+time_period+'_AUS_cont']]
data_n = pd.merge(data_SW_n,data_FR_n,on='code',how='outer')
data_n = pd.merge(data_AUS_n,data_n,on='code',how='outer')
data_n

In [ ]:
meta_analysis['odd_ratio_meta'] = meta_analysis.apply(lambda x : str(np.round(x['odd_ratio'],2)) + ' ('+ str(np.round(x['low_odd_ratio'],2))+'-'+str(np.round(x['high_odd_ratio'],2))+')',axis=1)
for country in ['AUS','FR','SW']: 
    if country == 'AUS':
        nb_tests = data_AUS.shape[0]
    elif country =='FR':
        nb_tests = data_FR.shape[0]
    else:
        nb_tests = data_SW.shape[0]
    counf_interval_95 = norm.ppf(1 - 0.025/nb_tests)   
    meta_analysis['odds_ratio_'+country] = meta_analysis.apply(lambda x : str(np.round(x['odds_ratio_'+country],2))+' ('+str(np.exp(np.log(x['odds_ratio_'+country])-counf_interval_95*x['std_'+country]).round(2))+'-'+str(np.exp(np.log(x['odds_ratio_'+country])+counf_interval_95*x['std_'+country]).round(2))+')',  axis=1)

meta_analysis = meta_analysis[['code','odd_ratio_meta','p_value','odds_ratio_SW','odds_ratio_FR','odds_ratio_AUS']]

mapping = pd.read_csv('mapping.csv',sep=';')
meta_analysis = pd.merge(meta_analysis,data_n,on='code',how='outer')
meta_analysis = pd.merge(meta_analysis,mapping,on='code',how='outer')
meta_analysis['final_name'] = meta_analysis['code']+': '+ meta_analysis['name_medoc']
meta_analysis = meta_analysis[['final_name','code','odd_ratio_meta','p_value','odds_ratio_SW','odds_ratio_FR','odds_ratio_AUS','n_'+time_period+'_AUS','n_'+time_period+'_AUS_cont','n_'+time_period+'_SW','n_'+time_period+'_SW_cont','n_'+time_period+'_FR2','n_'+time_period+'_FR_cont']]
meta_analysis = meta_analysis.replace('nan (nan-nan)','')
# Formater les p-values : arrondir à 3 décimales ou "< 0.001" si inférieur à 0.001
meta_analysis['p_value'] = meta_analysis['p_value'].apply(lambda x: '< 0.001' if x < 0.001 else round(x, 3))

meta_analysis = meta_analysis.sort_values(by=['final_name'])
meta_analysis.to_csv('meta_analysis_result_als_final_'+time_period+'.csv',sep=';')